# Client



# Secure Client Application

A C++ client application that establishes a secure SSL/TLS connection to a server, sends messages, and handles server responses using multi-threading and a custom logging mechanism.

## Table of Contents
- [Overview](#overview)
- [Architecture](#architecture)
- [Dependencies](#dependencies)
- [Installation](#installation)
- [Usage](#usage)
- [Code Structure](#code-structure)
- [Security Considerations](#security-considerations)
- [License](#license)

## Overview

This client application performs the following:
- Initializes SSL libraries.
- Creates and configures an SSL context.
- Establishes a TCP connection to a server.
- Performs an SSL handshake.
- Sends messages securely.
- Listens for server responses asynchronously.
- Logs activities and errors.

## Architecture

```plaintext
+------------------+
|    Initialize    |
|      SSL         |
+--------+---------+
         |
         v
+--------+---------+
| Create SSL Context|
+--------+---------+
         |
         v
+--------+---------+
|  Create Socket   |
|  and Connect     |
+--------+---------+
         |
         v
+--------+---------+
|  SSL Handshake   |
+--------+---------+
         |
         v
+--------+---------+       +-------------------------+
| Send Message     |<----->|  Listen for Responses   |
+--------+---------+       +-------------------------+
         |
         v
+--------+---------+
|  Handle Responses|
+--------+---------+
         |
         v
+--------+---------+
|  Clean Up and    |
|  Shutdown        |
+------------------+
```

## Dependencies

- **Operating System**: Unix-like (Linux, macOS)
- **Compiler**: GCC or Clang with C++11 support
- **Libraries**:
  - OpenSSL
  - POSIX Threads
  - Custom Logging (`quill::Logger`)

## Installation

### Install OpenSSL

```bash
sudo apt-get update
sudo apt-get install libssl-dev
```

### Clone Repository

```bash
git clone https://github.com/yourusername/secure-client.git
cd secure-client
```

### Build the Application

Ensure that `logger.h` and its implementation are in the project directory.

```bash
g++ -std=c++11 main.cpp -o client -lssl -lcrypto -lpthread
```

## Usage

```bash
./client
```

The client will:
1. Connect to the server at `192.168.2.2:8085`.
2. Perform an SSL handshake.
3. Send the message "Hello, server!".
4. Wait for a response or timeout after 5 seconds.
5. Log all activities and errors.

## Code Structure

### `main.cpp`

```cpp
#include <iostream>
#include <cstring>
#include <arpa/inet.h>
#include <sys/socket.h>
#include <openssl/ssl.h>
#include <openssl/err.h>
#include <unistd.h>
#include <chrono>
#include <thread>
#include <mutex>
#include <condition_variable>

#include "logger.h"

// Initialize SSL libraries
void initialize_ssl()
{
    SSL_load_error_strings();
    OpenSSL_add_ssl_algorithms();
}

// Create SSL context
SSL_CTX *create_context()
{
    const SSL_METHOD *method = TLS_client_method();
    SSL_CTX *ctx = SSL_CTX_new(method);
    if (!ctx)
    {
        std::cerr << "Unable to create SSL context" << std::endl;
        ERR_print_errors_fp(stderr);
        exit(EXIT_FAILURE);
    }
    return ctx;
}

// Listen for server responses
void listen_for_responses(quill::Logger *logger, SSL *ssl, bool &run, std::mutex &mtx, std::condition_variable &cv)
{
    while (run)
    {
        char buf[1024] = {0};
        int bytes = SSL_read(ssl, buf, sizeof(buf));

        std::lock_guard<std::mutex> lock(mtx);
        if (bytes > 0)
        {
            LOG_INFO(logger, "Server responded: {}", buf);
        }
        else if (bytes == 0)
        {
            LOG_INFO(logger, "Connection closed by the server.");
            run = false;
        }
        else
        {
            LOG_ERROR(logger, "SSL read error or connection lost.");
            run = false;
        }
        cv.notify_one();
    }
}

int main()
{
    initialize_ssl();
    SSL_CTX *ctx = create_context();

    quill::Logger *logger = initialize_logger();

    int sock = socket(AF_INET, SOCK_STREAM, 0);
    sockaddr_in addr;
    memset(&addr, 0, sizeof(addr));
    addr.sin_family = AF_INET;
    addr.sin_port = htons(8085);
    inet_pton(AF_INET, "192.168.2.2", &addr.sin_addr);

    if (connect(sock, (struct sockaddr *)&addr, sizeof(addr)) != 0)
    {
        perror("Connection error");
        exit(EXIT_FAILURE);
    }

    SSL *ssl = SSL_new(ctx);
    SSL_set_fd(ssl, sock);

    if (SSL_connect(ssl) == -1)
    {
        ERR_print_errors_fp(stderr);
    }
    else
    {
        LOG_INFO(logger, "Connected with {} encryption.", SSL_get_cipher(ssl));

        bool run = true;
        std::mutex mtx;
        std::condition_variable cv;
        std::thread response_thread(listen_for_responses, logger, ssl, std::ref(run), std::ref(mtx), std::ref(cv));

        const char *msg = "Hello, server!";
        SSL_write(ssl, msg, strlen(msg));

        LOG_DEBUG(logger, "Message sent: {}", msg);

        // Wait for a response or a timeout
        std::unique_lock<std::mutex> lock(mtx);
        if (cv.wait_for(lock, std::chrono::seconds(5)) == std::cv_status::timeout)
        {
            LOG_ERROR(logger, "Timeout waiting for server response.");
        }

        run = false;
        response_thread.join();
    }

    SSL_free(ssl);
    close(sock);
    SSL_CTX_free(ctx);
    return 0;
}
```

## Security Considerations

- Ensure that the server's IP and port are correctly configured.
- Validate SSL certificates as needed to prevent man-in-the-middle attacks.
- Handle sensitive data securely and avoid logging confidential information.